<a href="https://colab.research.google.com/github/KHAYYAM-SHAH/kunhar-river-climate-prediction/blob/main/ClientRiverKunhar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from google.colab import files
import io
import warnings
import re
warnings.filterwarnings('ignore')

def train_and_predict(df, features, target, future_df):
    scaler_X = StandardScaler()
    scaler_y = StandardScaler()

    X = df[features]
    y = df[target]

    X_scaled = scaler_X.fit_transform(X)
    y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1)).ravel()

    svr = SVR(kernel='rbf', C=100, gamma='auto', epsilon=0.1)
    svr.fit(X_scaled, y_scaled)

    X_future = future_df[features]
    X_future_scaled = scaler_X.transform(X_future)
    y_future_scaled = svr.predict(X_future_scaled)
    y_forecast = scaler_y.inverse_transform(y_future_scaled.reshape(-1, 1)).ravel()

    return y_forecast

def load_climate_data():
    print("📂 Upload 'cleaned_climate_data.csv'")
    uploaded = files.upload()
    df = pd.read_csv(io.BytesIO(uploaded[list(uploaded.keys())[0]]))
    try:
        df['DATE'] = pd.to_datetime(df['DATE'], format='%d/%m/%Y')
    except:
        df['DATE'] = pd.to_datetime(df['DATE'])
    df = df.fillna(method='ffill').fillna(df.mean(numeric_only=True))
    df['YEAR'] = df['DATE'].dt.year
    df['DOY'] = df['DATE'].dt.dayofyear
    return df[['DATE', 'YEAR', 'DOY', 'T2M', 'PRECTOTCORR', 'RH2M', 'WS2M']]

def load_discharge_data():
    print("📂 Upload discharge files (2009–2018)")
    uploaded = files.upload()
    dfs = []

    for name, content in uploaded.items():
        match = re.search(r'20\d{2}', name)
        year = int(match.group()) if match else None
        df = pd.read_csv(io.BytesIO(content)).dropna(how='all')

        day_col = [c for c in df.columns if c.strip().lower() == 'day']
        df['DAY'] = pd.to_numeric(df[day_col[0]], errors='coerce')
        month_cols = [c for c in df.columns if c not in day_col]

        df_melt = pd.melt(df, id_vars='DAY', value_vars=month_cols, var_name='MONTH', value_name='DISCHARGE')
        df_melt['MONTH'] = df_melt['MONTH'].str[:3].str.capitalize()
        month_map = {m: i + 1 for i, m in enumerate(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                                                     'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])}
        df_melt['MONTH'] = df_melt['MONTH'].map(month_map)
        df_melt['YEAR'] = year
        df_melt['DATE'] = pd.to_datetime(dict(year=df_melt['YEAR'], month=df_melt['MONTH'], day=df_melt['DAY']), errors='coerce')
        df_melt['DISCHARGE'] = pd.to_numeric(df_melt['DISCHARGE'], errors='coerce')
        df_melt = df_melt.dropna(subset=['DATE', 'DISCHARGE'])
        df_melt = df_melt.fillna(method='ffill').fillna(df_melt.mean(numeric_only=True))
        dfs.append(df_melt[['DATE', 'DISCHARGE']])

    final = pd.concat(dfs)
    final['DATE'] = pd.to_datetime(final['DATE']).dt.normalize()
    return final

def merge_data(climate_df, discharge_df):
    merged = pd.merge(climate_df, discharge_df, on='DATE', how='inner')
    return merged.fillna(merged.mean(numeric_only=True))

def generate_future_climate_data(climate_df):
    np.random.seed(42)
    climate_df['MONTH'] = climate_df['DATE'].dt.month
    monthly_avg = climate_df.groupby('MONTH')[['T2M', 'PRECTOTCORR', 'RH2M', 'WS2M']].mean().reset_index()
    future_dates = pd.date_range('2026-01-01', '2028-12-31')
    future_df = pd.DataFrame({'DATE': future_dates})
    future_df['MONTH'] = future_df['DATE'].dt.month
    future_df['YEAR'] = future_df['DATE'].dt.year
    future_df['DOY'] = future_df['DATE'].dt.dayofyear
    future_df = future_df.merge(monthly_avg, on='MONTH', how='left')

    future_df['T2M'] += (future_df['YEAR'] - 2025) * 0.1 + np.random.normal(0, 0.2, len(future_df))
    future_df['PRECTOTCORR'] += (future_df['YEAR'] - 2025) * 0.05 + np.random.normal(0, 0.1, len(future_df))
    future_df['RH2M'] += np.random.normal(0, 0.1, len(future_df))
    future_df['WS2M'] += np.random.normal(0, 0.1, len(future_df))

    return future_df

def plot_interactive(forecast_df, variable):
    forecast_df['MONTH'] = forecast_df['DATE'].dt.strftime('%b')
    monthly = forecast_df.groupby(['YEAR', 'MONTH'])[variable].mean().reset_index()
    months_order = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
    monthly['MONTH'] = pd.Categorical(monthly['MONTH'], categories=months_order, ordered=True)
    monthly = monthly.sort_values(['YEAR', 'MONTH'])

    color_map = {
        '2026': 'blue',
        '2027': 'orange',
        '2028': 'green'
    }

    fig = px.line(
        monthly, x='MONTH', y=variable, color='YEAR', markers=True,
        title=f'{variable} Forecast (2026–2028)',
        color_discrete_map=color_map,
        labels={variable: f'{variable} (m³/s)' if variable == 'DISCHARGE' else variable}
    )

    fig.update_layout(
        yaxis_title=f"{variable} (m³/s)" if variable == 'DISCHARGE' else variable,
        xaxis_title="Month",
        legend_title="Year"
    )

    fig.show()

def main():
    climate_df = load_climate_data()
    discharge_df = load_discharge_data()
    merged_df = merge_data(climate_df, discharge_df)

    future_df = generate_future_climate_data(climate_df)
    features = ['T2M', 'PRECTOTCORR', 'RH2M', 'WS2M']

    future_df['DISCHARGE'] = train_and_predict(merged_df, features, 'DISCHARGE', future_df)
    future_df['T2M'] = train_and_predict(merged_df, features, 'T2M', future_df)
    future_df['PRECTOTCORR'] = train_and_predict(merged_df, features, 'PRECTOTCORR', future_df)

    forecast_file = "/content/final_forecast_2026_2028.csv"
    future_df.to_csv(forecast_file, index=False)
    print(" Forecast saved to: final_forecast_2026_2028.csv")
    files.download(forecast_file)

    plot_interactive(future_df, 'DISCHARGE')
    plot_interactive(future_df, 'T2M')
    plot_interactive(future_df, 'PRECTOTCORR')

        # --- Forecast lookup for specific dates ---
    print("\n📅 Enter one or more dates (YYYY-MM-DD), separated by commas (e.g. 2026-05-01,2027-07-15):")
    input_str = input("➡️  Dates: ").strip()

    try:
        date_list = [pd.to_datetime(d.strip()) for d in input_str.split(',')]
        matched = future_df[future_df['DATE'].isin(date_list)][['DATE', 'DISCHARGE', 'T2M', 'PRECTOTCORR']]

        if matched.empty:
            print("❌ No forecast available for the entered date(s). Make sure they are between 2026-01-01 and 2028-12-31.")
        else:
            matched = matched.sort_values('DATE')
            print("\n🔍 Forecasted Values:")
            print(matched.to_string(index=False,
                                    header=['📅 Date', '🌊 Discharge (m³/s)', '🌡 Temp (°C)', '☔ Precipitation (mm)']))
    except Exception as e:
        print("⚠️ Invalid input. Please make sure dates are in YYYY-MM-DD format.")


if __name__ == "__main__":
    main()


📂 Upload 'cleaned_climate_data.csv'


Saving cleaned_climate_data.csv to cleaned_climate_data.csv
📂 Upload discharge files (2009–2018)


Saving kunhar_discharge_2009.csv to kunhar_discharge_2009.csv
Saving kunhar_discharge_2010.csv to kunhar_discharge_2010.csv
Saving kunhar_discharge_2011.csv to kunhar_discharge_2011.csv
Saving kunhar_discharge_2012.csv to kunhar_discharge_2012.csv
Saving kunhar_discharge_2013.csv to kunhar_discharge_2013.csv
Saving kunhar_discharge_2014.csv to kunhar_discharge_2014.csv
Saving kunhar_discharge_2015.csv to kunhar_discharge_2015.csv
Saving kunhar_discharge_2016.csv to kunhar_discharge_2016.csv
Saving kunhar_discharge_2017.csv to kunhar_discharge_2017.csv
Saving kunhar_discharge_2018.csv to kunhar_discharge_2018.csv
 Forecast saved to: final_forecast_2026_2028.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


📅 Enter one or more dates (YYYY-MM-DD), separated by commas (e.g. 2026-05-01,2027-07-15):
➡️  Dates: 2026-1-1, 2027-1-1, 2028-1-1

🔍 Forecasted Values:
    📅 Date 🌊 Discharge (m³/s) 🌡 Temp (°C) ☔ Precipitation (mm)
2026-01-01         128.189093    8.445773             3.439929
2027-01-01         124.217838    8.370161             3.339914
2028-01-01         129.746981    8.572027             3.421299
